# Agents

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/extras/use_cases/more/agents/agents.ipynb)

## Use case 

LLM-based agents are powerful general problem solvers.

The [primary LLM agent components](https://lilianweng.github.io/posts/2023-06-23-agent/) include at least 3 things:

* `Planning`: The ability to break down tasks into smaller sub-goals
* `Memory`: The ability to retain and recall information
* `Tools`: The ability to get information from external sources (e.g., APIs)

Unlike LLMs simply connected to [APIs](/docs/use_cases/apis/apis), agents [can](https://www.youtube.com/watch?v=DWUdGhRrv2c):

* Self-correct
* Handle multi-hop tasks (several intermediate "hops" or steps to arrive at a conclusion)
* Tackle long time horizon tasks (that require access to long-term memory)

![Image description](/img/agents_use_case_1.png)

## Overview 

LangChain has [many agent types](/docs/modules/agents/agent_types/).

Nearly all agents will use the following components:
 
**Planning**
 
* `Prompt`: Can given the LLM [personality](https://arxiv.org/pdf/2304.03442.pdf), context (e.g, via retrieval from memory), or strategies for learninng (e.g., [chain-of-thought](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/#chain-of-thought-cot)).
* `Agent` Responsible for deciding what step to take next using an LLM with the `Prompt`

**Memory**

* This can be short or long-term, allowing the agent to persist information.

**Tools**

* Tools are functions that an agent can call.

But, there are some taxonomic differences:

* `Action agents`: Designed to decide the sequence of actions (tool use) (e.g., OpenAI functions agents, ReAct agents).
* `Simulation agents`: Designed for role-play often in simulated enviorment (e.g., Generative Agents, CAMEL).
* `Autonomous agents`: Designed for indepdent execution towards long term goals (e.g., BabyAGI, Auto-GPT).

This will focus on `Action agents`.


## Quickstart 

In [ ]:
! pip install langchain openai google-search-results

# Set env var OPENAI_API_KEY and SERPAPI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_env()

`Tools`

LangChain has [many tools](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/load_tools.py) for Agents that we can load easily.

Let's load search and a calcultor.

In [ ]:
# Tool
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

`Agent`

The [`OPENAI_FUNCTIONS` agent](/docs/modules/agents/agent_types/openai_functions_agent) is a good action agent to start with.

OpenAI models have been fine-tuned to recognize when function should be called.

In [4]:
# Prompt
from langchain.agents import AgentExecutor
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are a search assistant.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

# Agent
search_agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=search_agent, tools=tools, verbose=False)

# Run
agent_executor.run("How many people live in canada as of 2023?")

'As of 2023, the estimated population of Canada is approximately 39,858,480 people.'

Great, we have created a simple search agent with a tool!

Note that we use an agent executor, which is the runtime for an agent. 

This is what calls the agent and executes the actions it chooses. 

Pseudocode for this runtime is below:
```
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action
```

While this may seem simple, there are several complexities this runtime handles for you, including:

* Handling cases where the agent selects a non-existent tool
* Handling cases where the tool errors
* Handling cases where the agent produces output that cannot be parsed into a tool invocation
* Logging and observability at all levels (agent decisions, tool calls) either to stdout or LangSmith.


## Memory 

### Short-term memory

Of course, `memory` is needed to enable conversation / persistence of information.

LangChain has many options for [short-term memory](/docs/modules/memory/types/), which are frequently used in [chat](/docs/modules/memory/adding_memory.html). 

They can be [employed with agents](/docs/modules/memory/agent_with_memory) too.

`ConversationBufferMemory` is a popular choice for short-term memory.

We set `MEMORY_KEY`, which can be referenced by the prompt later.

Now, let's add memory to our agent.

In [23]:
# Memory 
from langchain.memory import ConversationBufferMemory
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

# Prompt w/ placeholder for memory
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder
system_message = SystemMessage(content="You are a search assistant tasked with using Serpapi to answer questions.")
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

# Agent
search_agent_memory = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt, memory=memory)
agent_executor_memory = AgentExecutor(agent=search_agent_memory, tools=tools, memory=memory, verbose=False)

In [24]:
agent_executor_memory.run("How many people live in Canada as of August, 2023?")

'As of August 2023, the estimated population of Canada is approximately 38,781,291 people.'

In [25]:
agent_executor_memory.run("What is the population of its largest provence as of August, 2023?")

'As of August 2023, the largest province in Canada is Ontario, with a population of over 15 million people.'

Looking at the [trace](https://smith.langchain.com/public/4425a131-ec90-4aaa-acd8-5b880c7452a3/r), we can what is happening:

* The chat history is passed to the LLMs
* This gives context to `its` in `What is the population of its largest provence as of August, 2023?`
* The LLM generates a function call to the search tool

```
function_call:
  name: Search
  arguments: |-
    {
      "query": "population of largest province in Canada as of August 2023"
    }
```

* The search is executed
* The results frum search are passed back to the LLM for synthesis into an answer

![Image description](/img/oai_function_agent.png)

### Long-term memory 

Vectorstores are great options for long-term memory.

In [24]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
embedding_size = 1536
embeddings_model = OpenAIEmbeddings()
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

### Going deeper 

* Explore projects using long-term memory, such as [autonomous agents](/docs/use_cases/autonomous_agents/autonomous_agents).

## Tools 

As mentioned above, LangChain has [many tools](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/load_tools.py) for Agents that we can load easily.

We can also define [custom tools](/docs/modules/agents/tools/custom_tools). For example, here is a search tool.

* The `Tool` dataclass wraps functions that accept a single string input and returns a string output.
* `return_direct` determines whether to return the tool's output directly. 
* Setting this to `True` means that after the tool is called, the `AgentExecutor` will stop looping.

In [29]:
from langchain.agents import Tool, tool
from langchain.utilities import GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper()
search_tool = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True,
    )
]

To make it easier to define custom tools, a `@tool` decorator is provided. 

This decorator can be used to quickly create a Tool from a simple function.

In [30]:
# Tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)
word_length_tool = [get_word_length]

### Going deeper

**Toolkits**

* Toolkits are groups of tools needed to accomplish specific objectives.
* [Here](/docs/integrations/toolkits/) are > 15 different agent toolkits (e.g., Gmail, Pandas, etc). 

Here is a simple way to think about agents vs the various chains covered in other docs:

![Image description](/img/agents_vs_chains.png)

## Agents

There's a number of [action agent types](docs/modules/agents/agent_types/) available in LangChain.

* [ReAct](/docs/modules/agents/agent_types/react.html): This is the most general purpose action agent using the [ReAct framework](https://arxiv.org/pdf/2205.00445.pdf), which can work with [Docstores](/docs/modules/agents/agent_types/react_docstore.html) or [Multi-tool Inputs](/docs/modules/agents/agent_types/structured_chat.html).
* [OpenAI functions](/docs/modules/agents/agent_types/openai_functions_agent.html): Designed to work with OpenAI function-calling models.
* [Conversational](/docs/modules/agents/agent_types/chat_conversation_agent.html): This agent is designed to be used in conversational settings
* [Self-ask with search](/docs/modules/agents/agent_types/self_ask_with_search.html): Designed to lookup factual answers to questions

### OpenAI Functions agent

As shown in Quickstart, let's continue with [`OpenAI functions` agent](/docs/modules/agents/agent_types/).

This uses OpenAI models, which are fine-tuned to detect when a function should to be called.

They will respond with the inputs that should be passed to the function.

But, we can unpack it, first with a custom prompt:

In [31]:
# Memory
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

# Prompt
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

Define agent:

In [32]:
# Agent 
from langchain.agents import OpenAIFunctionsAgent
agent = OpenAIFunctionsAgent(llm=llm, tools=word_length_tool, prompt=prompt)

Run agent:

In [33]:
# Run the executer, including short-term memory we created
agent_executor = AgentExecutor(agent=agent, tools=word_length_tool, memory=memory, verbose=False)
agent_executor.run("how many letters in the word educa?")

'There are 5 letters in the word "educa".'

### ReAct agent

[ReAct](https://arxiv.org/abs/2210.03629) agents are another popular framework.

There has been lots of work on [LLM reasoning](https://ai.googleblog.com/2022/05/language-models-perform-reasoning-via.html), such as chain-of-thought prompting.

There also has been work on LLM action-taking to generate obervations, such as [Say-Can](https://say-can.github.io/).

ReAct marries these two ideas:

![Image description](/img/ReAct.png)
 
It uses a charecteristic `Thought`, `Action`, `Observation` [pattern in the output](https://lilianweng.github.io/posts/2023-06-23-agent/).
 
We can use `initialize_agent` to create the ReAct agent from a list of available types [here](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/types.py):

```
* AgentType.ZERO_SHOT_REACT_DESCRIPTION: ZeroShotAgent
* AgentType.REACT_DOCSTORE: ReActDocstoreAgent
* AgentType.SELF_ASK_WITH_SEARCH: SelfAskWithSearchAgent
* AgentType.CONVERSATIONAL_REACT_DESCRIPTION: ConversationalAgent
* AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION: ChatAgent
* AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION: ConversationalChatAgent
* AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION: StructuredChatAgent
* AgentType.OPENAI_FUNCTIONS: OpenAIFunctionsAgent
* AgentType.OPENAI_MULTI_FUNCTIONS: OpenAIMultiFunctionsAgent
```

In [41]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)
react_agent = initialize_agent(search_tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False, memory=memory)

In [ ]:
react_agent("How many people live in Canada as of August, 2023?")

In [ ]:
react_agent("What is the population of its largest provence as of August, 2023?")

LangSmith can help us run diagnostics on the ReAct agent:

The [ReAct agent](https://smith.langchain.com/public/3d8d0a15-d73f-44f3-9f81-037f7031c592/r) fails to pass chat history to LLM, gets wrong answer.
 
The OAI functions agent does and [gets right answer](https://smith.langchain.com/public/4425a131-ec90-4aaa-acd8-5b880c7452a3/r), as shown above.
 
Also the search tool result for [ReAct](https://smith.langchain.com/public/6473e608-fc9d-47c9-a8a4-2ef7f2801d82/r) is worse than [OAI](https://smith.langchain.com/public/4425a131-ec90-4aaa-acd8-5b880c7452a3/r/26b85fa9-e33a-4028-8650-1714f8b3db96).

Collectivly, this tells us: carefully inspect Agent traces and tool outputs. 

As we saw with the [SQL use case](/docs/use_cases/sql), `ReAct agents` can be work very well for specific problems. 

But, as shown here, the result is degraded relative to what we see with the OpenAI agent.

### Custom

Let's peel it back even further to define our own action agent.

We can [create a custom agent](/docs/modules/agents/how_to/custom_agent.html) to unpack the central pieces:

* `Tools`: The tools the agent has available to use
* `Agent`: decides which action to take

In [34]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent

class FakeAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")
    
fake_agent = FakeAgent()
fake_agent_executor = AgentExecutor.from_agent_and_tools(agent=fake_agent, 
                                                         tools=search_tool, 
                                                         verbose=False)

fake_agent_executor.run("How many people live in canada as of 2023?")

"The current population of Canada is 38,808,843 as of Tuesday, August 1, 2023, based on Worldometer elaboration of the latest United Nations data 1. Canada 2023\xa0... Mar 22, 2023 ... Record-high population growth in the year 2022. Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population\xa0... Jun 19, 2023 ... As of June 16, 2023, there are now 40 million Canadians! This is a historic milestone for Canada and certainly cause for celebration. It is also\xa0... Jun 28, 2023 ... Canada's population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023. The main driver of population growth is immigration, and to a lesser extent, natural growth. Demographics of Canada · Population pyramid of Canada in 2023. May 2, 2023 ... On January 1, 2023, Canada's population was estimated to be 39,566,248, following an unprecedented increase of 1,050,110 people between January\xa0... Canada ranks 37th by population

## Runtime

The `AgentExecutor` class is the main agent runtime supported by LangChain. 

However, there are other, more experimental runtimes for `autonomous_agents`:
 
* Plan-and-execute Agent
* Baby AGI
* Auto GPT

Explore more about:

* [`Simulation agents`](/docs/modules/agents/agent_use_cases/agent_simulations): Designed for role-play often in simulated enviorment (e.g., Generative Agents, CAMEL).
* [`Autonomous agents`](/docs/modules/agents/agent_use_cases/autonomous_agents): Designed for indepdent execution towards long term goals (e.g., BabyAGI, Auto-GPT).

